# Import packages

In [6]:
from bs4 import BeautifulSoup 
import re
import urllib
import requests
import shutil 
from selenium import webdriver
import os
import time
import copy
from tqdm import tqdm
from pykospacing import spacing
import pandas as pd
import copy
import string, random ## generate random str package
from collections import OrderedDict ## repetition removal package

# Function Definition

In [1]:
def hangul(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
    result = hangul.sub('', text) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    return str(result)

def find_categories():
    test = driver.find_elements_by_css_selector("body > div.lyr_category_lst1 > ul > li:nth-child(1) > ul > li > a")
    Categories = []
    re.sub(,test)
    for i in test:
        Categories.append(i.text.split("\n")[:2])
    Count_Categories = len(Categories)
    Categories
    return Categories, Count_Categories

def find_post_id():
    post = driver.find_element_by_css_selector('#rego_cover > div.cover_cont > div.tit_area > div.bloger > span.thumb > a')
    r = re.compile('logNo=.*')
    string = r.findall(post.get_attribute('ng-href'))
    return re.sub("logNo=",'',"".join(string))

def random_id():
    passkey='' # an empty str key
    for x in range(10): # length of the random passkeys

        if random.choice([1,2]) == 1:
            passkey += passkey.join(random.choice(string.ascii_letters)) # upper & lower cased letter
        else:
            passkey += passkey.join(random.choice(string.digits)) # numbers
    return passkey

def Spacing_text(text_list):
    spacing_list = []
    for i in text_list:
        if len(i) < 197:
            spacing_list.append(spacing(i))
        else:
            iteration = int(len(i) / 197)
            mod = len(i) % 197
            start = 0
            end = 197
            check = 0
            while True:
                # 시행횟수 < 몫
                if check < iteration:
                    spacing_list.append(spacing(i[start:end]))
                    start+=197
                    end+=197
                    check +=1
                else:
                    # 마지막 횟수 + 나머지 더 slice 
                    spacing_list.append(spacing(i[iteration*197:(iteration*197)+mod]))
                    break
    return spacing_list

def remove_odd(x):
    x = re.sub("&nbsp"," ",x)
    x = re.sub("nbsp"," ",x)
    x = re.sub("\xa0","",x)
    x = re.sub("\u200b","",x)
    x = re.sub("\n","",x)
    x = re.sub("\t","",x)
    x = re.sub("\r","",x)
    x = re.sub('   ',' ',x)
    return x.strip()

SyntaxError: invalid syntax (<ipython-input-1-44bf44956935>, line 10)

In [8]:
def Post_Id_Get_1():
    foot_info = driver.find_element_by_class_name('wrap_postcomment').get_attribute('innerHTML')
    p = re.compile('id="area_sympathy.*" ')
    post_id_string = p.findall(foot_info)
    if post_id_string == []:
        p = re.compile('id="Comi.*" ')
        post_id_string = p.findall(foot_info)
    remove_quotes = re.sub('"','',str(post_id_string[0]))
    post_no = "".join(list(filter(str.isdigit,remove_quotes)))
    return post_no

In [9]:
def Post_Id_Get_2():
    #게시글 번호 따오기

    foot_info = driver.find_element_by_id('postViewArea').get_attribute('innerHTML')
    p = re.compile('postViewArea.*"')
    post_id_string = "".join(p.findall(foot_info))

    post_id_string = re.sub('postViewArea','',post_id_string)
    post_id_string = re.sub('"','',post_id_string)
    return post_id_string

In [10]:
def Comment_Sympath_Count():
    p = re.compile('[0-9]')
    sympathy_count = ''
    comment_count = ''
    footer_list = driver.find_elements_by_class_name('wrap_postcomment')[0].text.split('\n')
    for foot in footer_list:
        if '공감' in foot:
            sympathy_count = "".join(p.findall(foot))      
        if '댓글' in foot:
            comment_count = "".join(p.findall(foot))

    if sympathy_count == '':
        sympathy_count = 0
    if comment_count == '':
        comment_count = 0
    return sympathy_count, comment_count

# Make csv_file

In [11]:
# user_csv 작성
csv_name = 'test_table.csv'
col_name = "\t".join(["User_id","Blog_name","Blog_nickname","Blog_info_text","Count_neighbors","Count_visitors","Categories","Count_categories","Total_post","Credibility","Source","Blog_mobile_profile_img","Blog_mobile_cover_img",'Opening_date'])

if not os.path.exists(csv_name):
    with open(csv_name, 'w') as f:
        f.write(col_name+'\n')
        
# post_csv 작성
post_csv_name = 'test_post_table.csv'
post_col_name = "\t".join(["Post_id","User_id","Category","Title","Date","Structure","Structure_tag","Text","Space_text","Count_space_mistake","Map_exist","Media_count","Heart_count","Sticker_count","Comment_count"])
if not os.path.exists(post_csv_name):
    with open(post_csv_name, 'w') as f:
        f.write(post_col_name +'\n')

# img_csv 작성
img_csv_name = 'test_img_table.csv'
img_col_name = "\t".join(["Img_id","Post_id"])
if not os.path.exists(img_csv_name):
    with open(img_csv_name, 'w') as f:
        f.write(img_col_name +'\n')

# tag_csv 작성
tag_csv_name = 'test_tag_table.csv'
tag_col_name = "\t".join(["Post_id","Post_tag"])
if not os.path.exists(tag_csv_name):
    with open(tag_csv_name, 'w') as f:
        f.write(tag_col_name +'\n')

# Set path

In [12]:
## Category 폴더 만들기

Category_list = ["와인·술","육아·결혼","등산·낚시·레저","드라마·방송","교육·학문","건강·의학","차·커피·디저트",\
"국내여행","어학·외국어","만화·애니","IT·컴퓨터","자동차","시사·인문·경제","패션·뷰티","공연·전시","스포츠","맛집","사진",'무작위']

Base_path = os.getcwd()
Category_path = "C:\\Users\\AjouHCI\\Desktop\\test\\dbdbdeep"
os.chdir(Category_path)
    
for directory in Category_list:
    if not os.path.exists(directory):
        os.makedirs(directory)
os.chdir(Base_path)

# 기본 path설정
dbdbdeep_folder = "C:\\Users\\AjouHCI\\Desktop\\test\\dbdbdeep\\" 
Base_path = 'C:\\workspace\\Credibility_Research'
Chrome_path = './/exe_file//chromedriver.exe'

# Post_count_number 입력
- sample 
- "http://www.netcury.com" "넷큐리로부터 지원받아 작성하였습니다"

In [13]:
start_number = 0
end_number = 20

In [ ]:
for post_count_number in tqdm(range(start_number,end_number)):
    print("post number:",post_count_number)
# Extract User_id and Post_id
#------------------------------------------------------------------------------------------------------------------------------------------------------------
     # Web blog 검색
    driver = webdriver.Chrome(Chrome_path)
    driver.get("https://search.naver.com/search.naver?where=post&sm=tab_jum")
    driver.implicitly_wait(3)

    url_list = []

    # Category List 나열

    for num,category in enumerate(Category_list):
        print(num,'\t',category)

    Category = input("Category를 입력하세요: ")
    check = 1

    while check:
        if Category not in Category_list:
            Category = input("Category를 다시 입력하세요: ")
        else:
            print("Category :",Category)
            check = 0

    # click blog in current page and save to list

    first_page = 1
    check = 1
    while check:
        if first_page == 1:
            first_page = 0
            urls = driver.find_elements_by_class_name("url")
            for num, url in enumerate(urls):
                url.click()   
                window_before = driver.window_handles[0]
                window_after = driver.window_handles[1]
                driver.switch_to_window(window_after)
                driver.implicitly_wait(10)
                url_list.append(driver.current_url)
                driver.close()
                driver.switch_to_window(window_before)

        # next_button click
        try:
            driver.find_element_by_class_name("next").click()
            driver.implicitly_wait(3)
            urls = driver.find_elements_by_class_name("url")

            for num, url in enumerate(urls):
                url.click()
                window_before = driver.window_handles[0]
                window_after = driver.window_handles[1]
                driver.switch_to_window(window_after)
                driver.implicitly_wait(10)
                url_list.append(driver.current_url)
                driver.close()
                driver.switch_to_window(window_before)

                if len(url_list) != len(set(url_list)):
                    url_list = url_list[:-1]
                    check = 0
                    break
        except:
            break

    print("Category: ",Category,"\t블로그 개수: ",len(url_list))

    # Get user_id and post_id from url

    User_id = []
    Post_id = []

    for url in url_list:

        if '?' in url: 
            User_id.append(re.sub("https://blog.naver.com/","",url).split("?")[0])
            Post_id.append("".join(re.findall("\d+",re.sub("https://blog.naver.com/","",url).split("?")[1])))

        elif 'blog.me' in url:
            url_temp = re.sub("https://","",url).split(".")
            temp_Post_id = url
            User_id.append(url_temp[0])
            Post_id.append(re.sub("https://blog.naver.com/","",temp_Post_id).split("/")[-1])

        else:
            url_temp = re.sub("https://blog.naver.com/","",url).split("/")
            User_id.append(url_temp[0])
            Post_id.append(url_temp[1])

    driver.close()
    
    ## Path 설정
    move_path =  dbdbdeep_folde

## User_Table Crwaling
#------------------------------------------------------------------------------------------------------------------------------------------------------------
    # user_id, post_id zip, remove_duplicated

    blog_list = list(set((zip(copy.copy(User_id),copy.copy(Post_id)))))
    blog_list = list(filter(lambda x : len(x[1]) > 0,blog_list))
    
    User_id = []
    Post_id = []
    
    # verifying blog_list
    for user_id, post_id in blog_list:
        url = "http://blog.naver.com/PostView.nhn?blogId="+user_id+ "&logNo=" + post_id +"&redirect=Dlog&widgetTypeCall=true"
        r = requests.get(url)
        if '접근하고자하는 블로그 아이디가 없습니다.' not in r.text or '해당 블로그가 없습니다.' not in r.text:
            User_id.append(user_id)
            Post_id.append(post_id)
    # blog_list _ verification        
    blog_list = list(set((zip(copy.copy(User_id),copy.copy(Post_id)))))
    blog_list = list(filter(lambda x : len(x[1]) > 0,blog_list))

    # source credibility
    Credibility = 0
    Source = "dbdbdeep"
        
    for user_id, post_id in blog_list:
        User_id = user_id
        Post_id = post_id
        Source = Source 
        Credibility = Credibility
        url = "http://blog.naver.com/PostView.nhn?blogId="+User_id+ "&logNo=" + Post_id +"&redirect=Dlog&widgetTypeCall=true"
        mobile_url = "http://m.blog.naver.com/PostView.nhn?blogId="+ User_id

        # get mobile_url
        driver = webdriver.Chrome(Chrome_path)
        driver.get(mobile_url)
        driver.implicitly_wait(3)

        # 존재하지 않는 게시물 클릭
        driver.find_element_by_class_name("btn_area").click()
        driver.implicitly_wait(10)
        time.sleep(1)
        # Blog_name, Blog_nickname, Blog_mobile_profile_img, Blog_info_text
        Blog_name = driver.find_element_by_css_selector('#rego_cover > div.cover_cont > div.tit_area > h2 > a > span').text
        Blog_nickname =driver.find_element_by_class_name("user_name").text

        try:
            post = driver.find_element_by_css_selector('#rego_cover > div.cover_cont > div.tit_area > div.bloger > span.thumb > a')
            Blog_mobile_profile_img_url = post.get_attribute('ng-href')  
        except:
            post = driver.find_element_by_css_selector('#rego_cover > div.cover_cont > div.tit_area > div.bloger > span.thumb > a')
            Blog_mobile_profile_img_url = post.get_attribute('href')
            
        try:
            Blog_info_text = driver.find_element_by_class_name("text").text

        except:
            Blog_info_text = ""
        Blog_info_text = remove_odd(Blog_info_text)    

        # Count_neighbors
        neighbors_string = re.sub(",","",driver.find_element_by_class_name("count_buddy").text)
        Count_neighbors = int(re.findall('\d+', neighbors_string)[0])

        # Count_visitors
        visitor_stirng = driver.find_elements_by_class_name('count')[0].text
        Count_visitors = re.sub(",","",visitor_stirng.split("전체")[1]).strip()

        # download_img
        Mobile_cover_img_url = driver.find_element_by_class_name("cover_img").get_attribute('bg-lazy-img')

        # category 버튼 클릭
        driver.find_element_by_css_selector("#rego_cover > div.cover_cont > div.btn_area > div > div:nth-child(2) > a > span.txt").click()
        driver.implicitly_wait(3)

        # Total_Post, Categories, Count_Categories, 

        Categories, Count_categories =  find_categories()
        Total_post = driver.find_element_by_class_name("num").text
        driver.close()
        
        Opening_date = Opening_Date_Get(user_id)
        

        os.chdir(move_path)

        # make category_folder
        folder_name = User_id
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        
        os.chdir(os.join(move_path,User_id))

        Save_path = dbdbdeep_folder '\\' + folder_name
        os.chdir(Save_path)
        cover_img = "Cover_" + User_id + ".jpg"
        Blog_mobile_profile_img = User_id + "\\" + cover_img

        profile_img = "Profile_" + User_id + ".jpg"
        Blog_mobile_cover_img =  User_id +  "\\" + profile_img
        
        #download_img
        profile_cover_image_check = os.listdir(os.join(move_path,User_id))
        print(profile_cover_image_check)
        if "Profile_" or "Cover_" not in profile_cover_image_check: 
            urllib.request.urlretrieve(Mobile_cover_img_url,cover_img)
            urllib.request.urlretrieve(Blog_mobile_profile_img_url,profile_img)

        os.chdir(Base_path)

        save_content_list = [User_id,Blog_name,Blog_nickname,Blog_info_text,Count_neighbors,Count_visitors,Categories,Count_categories,Total_post,Credibility,Source,Blog_mobile_profile_img,Blog_mobile_cover_img,Opening_date]
        save_content = "\t".join(list(map(str,save_content_list)))

        with open(csv_name, 'a', encoding='utf-8') as f:
            f.write(save_content+"\n")

# Post & Tag & Img Table_Crwaling        
#------------------------------------------------------------------------------------------------------------------------------------------------------------

    for user_id, post_id in blog_list:
        User_id = user_id
        Post_id = post_id
        Category = Category

        url = "http://blog.naver.com/PostView.nhn?blogId="+User_id+ "&logNo=" + Post_id +"&redirect=Dlog&widgetTypeCall=true"
        r = requests.get(url)
        bs = BeautifulSoup(re.sub('&nbsp;',' ',r.text).encode("utf-8"), "html.parser")

        #title
        Title = bs.find("h3", {"class": "se_textarea"})
        if (Title == None):
            Title = bs.find("span", {"class": "pcol1 itemSubjectBoldfont"})
        if (Title != None):
            Title = Title.text.strip()
        else:
            Title = "TITLE ERROR"

        #date
        # Append_value
        Date = bs.find("span", {"class": "se_publishDate pcol2 fil5"})

        if Date == None:
            Date = bs.find("p",{"class":"date fil5 pcol2 _postAddDate"})
        if Date == None:
            Date = bs.find("span",{"class":"se_publishDate pcol2"})

        Date_text = re.sub("\n","",Date.text)
        Date = re.sub("\t","",Date_text)

        #Tag, Content_structure, Text

        # structure
        structure = bs.find("div", {"id": "postViewArea"})
        if structure == None:
            structure = bs.find("div",{"class","se_component_wrap sect_dsc __se_component_area"})
        structure_tag = structure.find_all(['p','img'])


        # only tag & texf extract
        tag_list = []
        structure_list = []
        text_list = []

        for i in structure_tag:
            # p_tag만 불러오기
            if "<p" in (str(i)):
                tag_list.append('<p>')
                structure_list.append('<p>')
                # img만 있을 때

                if '<img' in str(i):
                    for j in i:
                        try:
                            if len(j.text)>1:
                                tag_list.append('<br>')
                                structure_list.append('<br>')
                                structure_list.append(j.text)
                                text_list.append(j.text)
                        except:
                            pass

                # img가 아닌 경우 span tag가 더 있을 때
                elif '<span' in str(i):
                    for j in i:
                        if '<br' in str(j):
                            structure_list.append(j.text)
                            text_list.append(j.text)
                            # br_tag가 2개 이상 있을 때

                            if len(j.findAll('br'))>2:
                                for _ in range(0,len(j.findAll('br'))):
                                    tag_list.append('<br>')
                                    structure_list.append('<br>')

                            # br_tag가 1개 있을 때
                            else:
                                tag_list.append('<br>')
                                structure_list.append('<br>')

                        # span은 있지만 br tag가 없을 때       
                        else:
                            try:
                                structure_list.append(j.text)
                                text_list.append(j.text)
                            except:
                                pass

                # 그냥 p_tag만 있을 때 br_tag 추가
                else:
                    # 글이 있을 때
                    if len(i.text) > 1:
                        structure_list.append(i.text)
                        text_list.append(i.text)

                    # 글 없이 br tag만 있을 때
                    else:
                        tag_list.append('<br>')
                        structure_list.append(i.text)
                        structure_list.append('<br>')
                        text_list.append(i.text)

                # P_tag 끝맽음      
                tag_list.append('</p>')
                structure_list.append('</p>')

            else:
                tag_list.append('<img>')
                structure_list.append('<img>')


        structure_list = list(map(remove_odd,structure_list))
        text_list = list(map(remove_odd,text_list))
        filter_text = list(filter(lambda x: len(x)>1 ,text_list))

        Structure = "|".join(list(filter(lambda x: len(x)>1 ,structure_list)))        
        Text = " ".join(list(filter(lambda x: len(x)>1 ,map(lambda x : x.strip(),text_list))))
        Text = re.sub('\n','',Text)
        Text = re.sub('\t','',Text)
        Space_text = " ".join(Spacing_text(filter_text))
        Count_space_mistake = len(Space_text)-len(Text)


        # only tag
        Structure_tag = "|".join(tag_list)

        # image download    
        imgs = structure.find_all('img')
        Save_path = dbdbdeep_folder + Category + '\\' + User_id


        Map_exist = 0
        Sticker_count = 0
        # Sticker_count로 수정
        sticker_img = structure.find_all('a')
        for i in sticker_img:
            if 'sticker' in str(i):
                Sticker_count += 1
        
        # image download    
        imgs = structure.find_all('img')
        
        for img in imgs:

            if "스티커 이미지" in str(img):
                img_name = "Sticker_" + User_id + "_" + random_id() + ".jpg"
            else:
                img_name =  User_id + "_" + random_id() + ".jpg"
            try:
                img_url = re.sub("\u200b","",str(img['src']))

                try:
                    urllib.request.urlretrieve(img_url,img_name)
                    shutil.move(img_name,Save_path)

                except UnicodeEncodeError:
                    try:
                        if 'map' not in str(img_url):
                            driver = webdriver.Chrome(Chrome_path)
                            driver.get(url)
                            driver.implicitly_wait(3)
                            img = driver.find_element_by_tag_name('img')
                            src = re.sub("\u200b","",str(img.get_attribute('src')))
                            urllib.request.urlretrieve(src, Save_path)
                            shutil.move(img_name,move_folder)
                            driver.close()
                        else:
                            Map_exist = 1
                    except:
                        pass
                except:
                    pass

                Img_id = "dbdbdeep" + '\\'+ Category + '\\' + User_id + '\\'+ img_name
                save_img_content = Img_id + "\t" + Post_id

                with open(img_csv_name, 'a', encoding='utf-8') as f:
                    f.write(save_img_content + "\n")
            except:
                pass

        # Post_tag, Media_count
        driver = webdriver.Chrome('.//exe_file//chromedriver.exe')
        driver.get(url)
        driver.implicitly_wait(10)
        
        #extract comment_count
        Heart_count, Comment_count = Comment_Sympath_Count()
        Media_count = 0
        Media_check = driver.find_elements_by_tag_name('iframe')

        for Media in Media_check:
            if 'Player' in str(Media.get_attribute('src')):
                Media_count += 1
        tag_list = re.sub('\n','',driver.find_element_by_class_name('wrap_tag').text).strip().split('#')[1:]
        
        for Post_tag in tag_list:
            save_tag = Post_id + "\t" + Post_tag
            with open(tag_csv_name, 'a', encoding='utf-8') as f:
                f.write(save_tag + "\n")
                
        for Media in Media_check:
            if 'Player' in str(Media.get_attribute('src')):
                Media_count += 1
        # make a save list
        save_post = "\t".join(list(map(str,[Post_id,User_id,Category,Title,Date,Structure,Structure_tag,Text,Space_text,Count_space_mistake,Map_exist,Media_count,Heart_count,Sticker_count,Comment_count])))

        with open(post_csv_name, 'a', encoding='utf-8') as f:
                f.write(save_post + "\n")
        driver.close()

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

post number: 0
0 	 와인·술
1 	 육아·결혼
2 	 등산·낚시·레저
3 	 드라마·방송
4 	 교육·학문
5 	 건강·의학
6 	 차·커피·디저트
7 	 국내여행
8 	 어학·외국어
9 	 만화·애니
10 	 IT·컴퓨터
11 	 자동차
12 	 시사·인문·경제
13 	 패션·뷰티
14 	 공연·전시
15 	 스포츠
16 	 맛집
17 	 사진
18 	 무작위
Category를 입력하세요: 드라마·방송
Category : 드라마·방송
Category:  드라마·방송 	블로그 개수:  51
